In [41]:
#Importing packages
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from dotenv import load_dotenv
import psycopg2

In [42]:
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

In [43]:
try:
    conn = psycopg2.connect(DATABASE_URL)
    print("Connected to the database")
    cursor = conn.cursor()
    sql = """ SELECT id,name FROM \"Tags\" """
    cursor.execute(sql)
    db_tags = cursor.fetchall()
    dict_tags = {}
    for i in db_tags:
        dict_tags[i[0]] = i[1]
        
    feature_list =["uuid"]+[i[0] for i in db_tags]
    df = pd.DataFrame(columns=feature_list)
    sql = """  SELECT \"poemId\",\"tagId\" FROM  \"PoemsTags\" """
    cursor.execute(sql)
    poems_tags = cursor.fetchall()
   
    dict_poems_tags = {}
    for i in poems_tags:
        if i[0] in dict_poems_tags:
            dict_poems_tags[i[0]][i[1]] = 1
        else:
            dict_poems_tags[i[0]] = {}
            dict_poems_tags[i[0]]['uuid'] = i[0]
            dict_poems_tags[i[0]][i[1]] = 1
    
    for i in dict_poems_tags:
        df.loc[len(df)] =   dict_poems_tags[i]
        

    df = df.fillna(0)
    #df.to_csv('model.csv',index=False)
except Exception as e:
    print("Database Error",e)
finally:
    conn.close()

Connected to the database


In [63]:
data =  df.drop(['uuid'],axis=1)
neigh = NearestNeighbors(n_neighbors=5, metric='cosine',algorithm='brute')
neigh.fit(data)

try:
    conn = psycopg2.connect(DATABASE_URL)
    print("Connected to the database")
    cursor = conn.cursor()
    
    for i in dict_poems_tags:
        post_data = {}
        for tag in dict_tags:
            if tag in dict_poems_tags[i]:
                post_data[tag] = 1
            else:
                post_data[tag] = 0
        post = pd.DataFrame(data=post_data,index=[0])
    
        distances,indices = neigh.kneighbors(post)
        
        for idx in range(len(distances.flatten())):
            #  print('{0}: {1}, with a distance of {2}.'.format(i+1, movies['Movie Name'].iloc[indices.flatten()[i]],distances.flatten()[i]))
            uuid = (df['uuid'].iloc[indices.flatten()[idx]])
            if uuid!=dict_poems_tags[i]['uuid']:
                sql = """ INSERT INTO \"Recommendations\" (\"postId\",\"postRecommendId\",\"score\") VALUES (%s,%s,%s) """
                values = (dict_poems_tags[i]['uuid'],uuid,float(distances.flatten()[idx]),)
                cursor.execute(sql,values)
                conn.commit()
                print(values)
except Exception as e:
    print("Error",e)
finally:
    conn.close()
    

Connected to the database
('2797d87b-a1d0-40fc-b584-545e82f5eb85', '19b61440-7578-4aa1-9179-4e825defea17', 0.6000000000000001)
('2797d87b-a1d0-40fc-b584-545e82f5eb85', 'c1bcac84-488d-4ee1-84a6-d0652af0efb7', 0.6000000000000001)
('2797d87b-a1d0-40fc-b584-545e82f5eb85', '8d1f633d-bf1c-450e-b2d1-bfd9df56d2b4', 0.6000000000000001)
('2797d87b-a1d0-40fc-b584-545e82f5eb85', '45a57632-d3f9-4a9a-9615-1e4606de420b', 0.6000000000000001)
('ab4a394e-d907-4631-a4c2-5691c01f4c2a', '740cddcd-b6c9-47ee-8c4c-082b47581bfd', 0.6000000000000001)
('ab4a394e-d907-4631-a4c2-5691c01f4c2a', '4c120369-1701-4738-a521-df84c2c63539', 0.6000000000000001)
('ab4a394e-d907-4631-a4c2-5691c01f4c2a', 'b72267e0-925f-4345-a528-d7663d1b53ea', 0.6000000000000001)
('ab4a394e-d907-4631-a4c2-5691c01f4c2a', 'c91ebf03-2af5-4220-8192-f212be25e077', 0.6000000000000001)
('ff30b09e-7ffb-472f-8e9c-7c3d2f6d43aa', 'bcaedafe-f97e-4d8a-848a-fc7aec801e75', 0.4)
('ff30b09e-7ffb-472f-8e9c-7c3d2f6d43aa', '6c67ea42-00bc-4d24-8123-c2b9c0dd51de',